In [2]:
from pyspark.sql import SparkSession   
from pyspark.sql.types import StructType, StructField, \
  StringType, IntegerType, FloatType, DateType

spark = SparkSession.builder \
  .appName('read') \
  .getOrCreate()

movie_schema = StructType([
    StructField('movie_id', IntegerType(), True),
    StructField('title', StringType(), True),
    StructField('genre', StringType(), True),
    StructField('release_date', DateType(), True),
    StructField('budget', FloatType(), True),
])

actor_schema = StructType([
    StructField('actor_id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('birth_date', DateType(), True),
    StructField('country', StringType(), True)
])

relation_schema = StructType([
    StructField('movie_id', IntegerType(), True),
    StructField('actor_id', IntegerType(), True)
])

df_movie = spark.read.csv('movies.csv', 
                    header=True, schema=movie_schema)
df_actor = spark.read.csv('actors.csv', 
                    header=True, schema=actor_schema)
df_relation = spark.read.csv('movie_actors.csv', 
                    header=True, schema=relation_schema)

"""
# Общая информация о наборах данных
df_movie.printSchema()
df_actor.printSchema()
df_relation.printSchema()

# print('Первые пару записей:')
df_movie.show()
df_actor.show()
df_relation.show()
"""

# Создание временных таблиц
df_movie.createOrReplaceTempView("movies")   
df_actor.createOrReplaceTempView("actors")
df_relation.createOrReplaceTempView("relations")

# 1
print('Топ-5 жанров по количеству фильмов')
spark.sql("""
    SELECT genre, COUNT(*) `count`
    FROM movies
    GROUP BY genre
    ORDER BY count DESC 
"""
).show(5)

# 2
print('Актер(ы) с наибольшим количеством фильмов')
spark.sql("""
    with t as (SELECT actor_id, COUNT(*) `count`
               FROM relations
               GROUP BY actor_id)
    
    SELECT name
    FROM actors
    WHERE actor_id IN (SELECT actor_id
                       FROM t t1
                       WHERE t1.count = (SELECT MAX(t2.count)
                                        FROM t t2))
"""
).show()

# 3
print('Средний бюджет фильмов по жанрам')
spark.sql("""
    SELECT genre, AVG(budget) `average budget`
    FROM movies
    GROUP BY genre
""").show()

# 4
print('Фильмы, в которых снялось более одного актера из одной страны')
spark.sql("""
    SELECT m.title, a.country, count(r.actor_id) `count`
    FROM movies m  
        JOIN relations r on m.movie_id = r.movie_id
        JOIN actors a on a.actor_id = r.actor_id
    GROUP BY m.title, a.country
    HAVING count > 1
""").show()

spark.stop()

Топ-5 жанров по количеству фильмов
+------+-----+
| genre|count|
+------+-----+
| Drama|    6|
|Action|    6|
|Comedy|    4|
|Horror|    2|
|Sci-Fi|    2|
+------+-----+

Актер(ы) с наибольшим количеством фильмов
+--------+
|    name|
+--------+
|Actor_17|
|Actor_24|
+--------+

Средний бюджет фильмов по жанрам
+------+--------------+
| genre|average budget|
+------+--------------+
| Drama|   6.0760218E7|
|Horror|   8.7281876E7|
|Comedy| 5.207096625E7|
|Action|2.7492741875E7|
|Sci-Fi|   7.8097152E7|
+------+--------------+

Фильмы, в которых снялось более одного актера из одной страны
+--------+---------+-----+
|   title|  country|count|
+--------+---------+-----+
| Movie_7|    India|    2|
| Movie_3|      USA|    2|
|Movie_10|       UK|    2|
|Movie_15|    India|    2|
|Movie_18|Australia|    2|
| Movie_1|    India|    3|
| Movie_2|      USA|    2|
| Movie_7|      USA|    2|
|Movie_10|      USA|    2|
+--------+---------+-----+

